In [1]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import matplotlib.pyplot as plt # MATPLOTLIB
import seaborn as sns # SEABORN
import scipy
import sklearn
import random as rd
from sklearn.decomposition import PCA
from IPython.display import display
import cv2
import os 
from pathlib import Path

In [ ]:
###Aca vamos a sacar los b de textura

In [2]:

subjects_images = '../ck+copy/extended-cohn-kanade-images/cohn-kanade-images'
sorted_subjects =   np.sort(os.listdir('../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/')) # we probably need to not have the sorted

images = []

for subject in sorted_subjects:
    one_subject_images = '../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/' + str(subject)+ '/'
    sorted_list = np.sort(os.listdir(one_subject_images)) #creo que este sort se puede quitar
    images_per_subject = []
    for emotion in sorted_list:
        images__per_seqframe = []
        subject_image_emotion = one_subject_images + str(emotion)
        for i in range(0, len(os.listdir(subject_image_emotion))):
            image = '../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/' +  str(subject)+ '/' + str(emotion)+'/'+ str(subject)+ '_'+str(emotion)+'_'+ str(i+1).zfill(8)+'.png'
            images__per_seqframe.append(image)        
        images_per_subject.append(images__per_seqframe)
    images.append(images_per_subject)   
       

In [3]:
for i in range(0, len(images)):
    for j in range(0, len(images[i])):
                images[i][j]= np.asarray( images[i][j], dtype=object)
    images[i]= np.asarray( images[i], dtype=object)
images= np.asarray( images, dtype=object)

In [4]:
images[0][0].shape

(29,)

In [5]:
#See if loading image works
img= cv2.imread(images[0][0][0])

img =cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_width, img_height = img.shape
print(img.shape)

cv2.imshow("Face landmark result", img)

# Pause screen to wait key from user to see result
cv2.waitKey(0)
cv2.destroyAllWindows()

(490, 640)


In [6]:
#por ahora usamos numeros enteros
left_eye_warp = np.array([ 0.3 * img_width, float(round(img_height / 3 ))]) 
right_eye_warp =np.array([ 0.7 * img_width , float(round(img_height / 3 ))])

In [7]:
#Equilateral triangle
def equilateral_point(left, right):
    X, Y = left, right
    M = (X + Y) / 2
    O = (X - M) * 3**0.5
    t = np.array([[0, -1], [1, 0]])   # 90 degree transformation matrix

    thrid_point = M + O @ t
    return thrid_point
#
#In []:
#M + O @ t

#Out[]:
#array([1.5      , 1.8660254])

#In []:
#M + O @ t.T                       # the transpose gives -90

#Out[]:
#array([1.5      , 0.1339746])

In [8]:
third_point = equilateral_point(left_eye_warp, right_eye_warp)
third_point

array([245.        , 382.74097914])

In [17]:
right_eye_warp

(343.0, 213.0)

In [ ]:
landmark_37(point 36) y landmark 46(45) son los corners de los ojos

In [9]:
my_landmarks = pd.read_csv('ckmock_landmark_intercalated_data.csv',header = None)

In [10]:
even = np.asarray(my_landmarks.iloc[0][::2] )
odds = np.asarray( my_landmarks.iloc[0][1::2] )



left_eye_corner = np.array([even[36], odds[36]])
right_eye_corner = np.array([even[45], odds[45]])


third_point_corner = equilateral_point(left_eye_corner, right_eye_corner)



In [11]:
rows = 490
cols = 640
input_pts =  np.float32([left_eye_corner, right_eye_corner, third_point_corner])
output_pts =  np.float32([left_eye_warp, right_eye_warp, third_point])
M = cv2.getAffineTransform(input_pts , output_pts)
 
# Apply the affine transformation using cv2.warpAffine()
dst = cv2.warpAffine(img, M, (cols,rows))
 
# Display the image
out = cv2.hconcat([img, dst])
cv2.imshow('Output', out)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
lan =[]
for (a, b) in zip(even, odds):
    lan.append([[a,b]])
    
#np.array([ [[x1, y1]], ..., [[xn, yn]] ])
#dst=(np.dot(M[:,:2], a.T)+M[:,2].reshape(2,1)).T
lan =np.asarray(lan)
transformed_landmarks = cv2.transform(lan, M)

In [ ]:
#Lets draw the landmarks in the image before and after the warp
for landmark in lan:
 


cv2.imshow('Output', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

for landmark in lan:
  po


cv2.imshow('Output', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
we warp all the training face images to a
reference shape using cubic polynomial function

In [ ]:
For that, we first load the landmarks of the procustres analysis
used before as the references points for the warp 
(we use the intercalated ones as for now)


In [ ]:
def align_to_shape(image, labels, target, extend=1.0, rotate=False):
  image = np.array(image)
  labels = np.array(labels).astype(np.float32)
  target = np.array(target).astype(np.float32)
  m, _ = cv2.estimateAffinePartial2D(labels, target)
  image_t = cv2.warpAffine(image, m, (128, 128))
  labels_t = np.dot(labels, m[:,:2].T) + m[np.newaxis,:,2]
  return image_t, labels_t

In [16]:
my_aligned_landmarks = pd.read_csv('ckmock_landmark_aligned_intercalated_data.csv',header = None)

In [23]:
my_aligned_landmarks.iloc[0]

0     -0.163450
1     -0.068005
2     -0.156895
3     -0.028732
4     -0.145752
         ...   
131    0.071911
132    0.002726
133    0.072451
134   -0.014327
135    0.071782
Name: 0, Length: 136, dtype: float64

In [ ]:
Ahora vamos a hacer el wrap de todas las imagenes

In [ ]:
Por ahora estoy usando la misma imagen que en procustres. Deberia usar generalizado para hallar la de menor diferencia y usar esta